In [3]:
!pip install -r requirements.txt
!python -m spacy download en_core_web_sm

  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)
  Preparing metadata (setup.py) ... done
     ---------------------------------------- 30.5/30.5 MB 42.4 MB/s eta 0:00:0000:0100:01
     -------------------------------------- 298.0/298.0 kB 77.2 MB/s eta 0:00:00
     ---------------------------------------- 33.7/33.7 MB 38.0 MB/s eta 0:00:0000:0100:01
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1304 sha256=c50c23c0c0f806d35925a1d0d7232c07ec69b0a5c43c26df00ceb30c60e67e31
  Stored in directory: /root/.cache/pip/wheels/95/8e/3e/6372bf25c0ed7afad69bc3612e851d17314f5a77655811fd99
Successfully built sklearn
     ---------------------------------------- 12.8/12.8 MB 62.2 MB/s eta 0:00:0000:010:01
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [3]:
!python3 dataset_utils.py

ObjectUse 140669
CapableOf 7216
MadeUpOf 2911
HasProperty 4971
Desires 2718
NotDesires 2807
AtLocation 17423
Causes 329
HinderedBy 77616
xReason 292
isAfter 16485
isBefore 17094
HasSubEvent 10894
isFilledBy 24174
xIntent 58997
xReact 65984
oReact 54632
xAttr 120447
xEffect 93413
xNeed 105557
xWant 110009
oEffect 65290
oWant 76952
1076880 / 1076880


In [3]:
!python3 gpt_finetune.py parameters.json

10/20/2022 13:24:54 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 2distributed training: False, 16-bits training: False
10/20/2022 13:24:54 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=2,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
h

In [21]:
!python gpt_finetune.py \
    --model_name_or_path gpt2 \
    --dataset_name wikitext \
    --dataset_config_name wikitext-2-raw-v1 \
    --per_device_train_batch_size 8 \
    --per_device_eval_batch_size 8 \
    --do_train \
    --do_eval \
    --output_dir /tmp/test-clm

10/19/2022 16:53:31 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 2distributed training: False, 16-bits training: False
10/19/2022 16:53:31 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=2,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
h

In [18]:
!python3 gpt_finetune.py --help

usage: gpt_finetune.py [-h] [--model_name_or_path MODEL_NAME_OR_PATH]
                       [--model_type MODEL_TYPE]
                       [--config_overrides CONFIG_OVERRIDES]
                       [--config_name CONFIG_NAME]
                       [--tokenizer_name TOKENIZER_NAME]
                       [--cache_dir CACHE_DIR]
                       [--use_fast_tokenizer [USE_FAST_TOKENIZER]]
                       [--no_use_fast_tokenizer]
                       [--model_revision MODEL_REVISION]
                       [--use_auth_token [USE_AUTH_TOKEN]]
                       [--dataset_name DATASET_NAME]
                       [--dataset_config_name DATASET_CONFIG_NAME]
                       [--train_file TRAIN_FILE]
                       [--validation_file VALIDATION_FILE]
                       [--max_train_samples MAX_TRAIN_SAMPLES]
                       [--max_eval_samples MAX_EVAL_SAMPLES]
                       [--block_size BLOCK_SIZE]
                       [--overwr

In [38]:
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed

model_name_or_path="./tmp/test-clm/checkpoint-1000"
#model_name_or_path='gpt2'
device='cuda'
set_seed(10)

model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
model.to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
input_ids = tokenizer.encode("Answer this question: Of which color is the black horse? The color of the horse is", return_tensors="pt")
generations = model.generate(input_ids=input_ids.to(device), do_sample=True, num_return_sequences=1, max_new_tokens=100)
for gen in generations:
    new_text = tokenizer.decode(gen)
    print(new_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Answer this question: Of which color is the black horse? The color of the horse is colora trombone can be used to teachPersonX loves PersonY's ___ as themselves. PersonX loves everyone.PersonX works as a substitute teacher. After, PersonX will want to  pay billsPersonX goes ___ on a date. PersonX is determinedPersonX spends most of PersonY's time. This would not happen if PersonY lives so far away that PersonX can't attend a date.PersonX leaves PersonY's room. PersonX is independentPersonX has


In [6]:
!python3 gpt_finetune.py parameters.json

10/20/2022 14:09:05 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 2distributed training: False, 16-bits training: False
10/20/2022 14:09:05 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=2,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
